#### Softmax custom MNIST
Softmax regression example for custom MNIST images (instead of TensorFlow native MNIST dataset)

In [1]:
import numpy as np
from PIL import Image
import os
import csv
import tensorflow as tf
# function to read images and labels from a CSV
# CSV format: <relative file path>,<label>
def get_dataset(csv_path):
  images = []
  labels = []
  
  with open(csv_path, 'rb') as csvfile:
    csv_reader = csv.reader(csvfile)
    for rel_path, label in csv_reader:
      # create absolute image path
      #abs_path = os.getcwd() + '/MNIST_data/mnist/' + rel_path
      abs_path = os.getcwd() + "/mnist/" + rel_path
      # read image and reshape from 28x28 to 1x784
      img = np.asarray(Image.open(abs_path)).reshape(1, -1)
      img_f32 = np.float32([x / 255.0 for x in img])
      # append images and labels to lists
      images.append(img_f32)
      labels.append(int(label))
  
  # convert lists into numpy ndarrays and reshape
  images_array = np.asarray(images).reshape(-1, 784)
  labels_array = np.asarray(labels).reshape(1, -1).flatten()
  # one-hot encode labels
  sess = tf.Session()
  with sess.as_default():
    labels_one_hot = tf.one_hot(labels_array, 10).eval()
  
  return images_array, labels_one_hot

train_images, train_labels = get_dataset(os.getcwd() + "/mnist/train-labels.csv")
test_images, test_labels = get_dataset(os.getcwd() + "/mnist/test-labels.csv")

In [2]:
# https://www.tensorflow.org/tutorials/mnist/beginners/
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(1000):
  # select a random minibatch (MNIST sample size: 60000, batch size: 100)
  indices = np.random.choice(60000, 100)
  batch_xs, batch_ys = train_images[indices], train_labels[indices]
  
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [3]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: test_images, y_: test_labels}))

0.92
